In [24]:
%%javascript 
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [25]:
%autoreload 2
%matplotlib inline 
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)

In [26]:
!du -h ../../data/*01_30_16.h5

4.6G	../../data/test_DM_fhat_clst5_01_30_16.h5
664K	../../data/test_DM_peaks_df_clst5_01_30_16.h5
 22M	../../data/test_stars_fhat_clst5_01_30_16.h5
 84K	../../data/test_stars_peak_df_clst5_01_30_16.h5


In [42]:
from __future__ import print_function
import pandas as pd
import h5py
import sys
sys.path.append("../")
import extract_catalog as ec
import get_DM_centroids as getDM
import get_gal_centroids as getgal
import seaborn as sns

In [28]:
import compute_distance as compDist
import plot_DM_prop as plotDM

In [29]:
sns.set_style('white')

In [30]:
data_path = "../../data/"

In [31]:
clstNo = 5
date_stamp = '01_30_16'
DM_df = \
    pd.read_hdf(data_path + 
                "test_DM_peaks_df_clst{0}_{1}.h5".format(clstNo, 
                                                         date_stamp), 
                "peak_df")
DM_fhat = \
    h5py.File(data_path + 
              "test_DM_fhat_clst{0}_{1}.h5".format(clstNo, 
                                                   date_stamp))

In [32]:
star_fhat_file = "test_stars_fhat_clst{0}_{1}.h5".format(
    clstNo, date_stamp)
star_fhat = \
    h5py.File(data_path + 
              "test_stars_fhat_clst{0}_{1}.h5".format(clstNo, 
                                                      date_stamp))

star_df = \
    pd.read_hdf(data_path + 
                "test_stars_peak_df_clst{0}_{1}.h5".format(
            clstNo, date_stamp),
                "peak_df")

# Visualize the smoothed and not smoothed histograms

In [45]:
gal_metadata = getgal.metadata_from_h5_key_path(path)

In [46]:
gal_metadata

OrderedDict([('phi', u'0.7853981634'),
             ('xi', u'0.4111378623'),
             ('cut', u'min'),
             ('weights', u'i_band'),
             ('clstNo', u'13'),
             ('los_axis', u'1')])

In [33]:
DM_gpBy_key_no = 8 
star_gpBy_key_no = len(star_df.keys()) - 3
DM_path_list = \
    compDist.retrieve_cluster_path(DM_fhat, DM_gpBy_key_no)

In [49]:
DM_path_list[0].split('/')

[u'13',
 u'min',
 u'i_band',
 u'1',
 u'0.4111378623',
 u'0.7853981634',
 u'0.20',
 u'0']

In [34]:
DM_gp_keys = tuple(DM_df.keys()[-9:])
gal_gp_keys = tuple(star_df.keys()[-6:])


In [41]:
DM_df.keys()

Index([u'peaks_xcoords', u'peaks_ycoords', u'peaks_dens', u'clstNo', u'cut', u'weights', u'los_axis', u'xi', u'phi', u'sig_fraction', u'kernel_width'], dtype='object')

In [35]:
DM_gp_keys

('peaks_dens',
 'clstNo',
 'cut',
 'weights',
 'los_axis',
 'xi',
 'phi',
 'sig_fraction',
 'kernel_width')

In [36]:
gal_gp_keys

('clstNo', 'cut', 'weights', 'los_axis', 'xi', 'phi')

In [37]:
DM_gpBy = DM_df.groupby(DM_gp_keys, as_index=False)

In [38]:
len(DM_gp_keys)

9

In [40]:
path

u'13/min/i_band/1/0.4111378623/0.7853981634/0.20/0'

In [ ]:
compDist.DM_h5path_to_groupbykey(path)

In [73]:
DM_fhat.visit(print)

13
13/min
13/min/i_band
13/min/i_band/1
13/min/i_band/1/0.4111378623
13/min/i_band/1/0.4111378623/0.0000000000
13/min/i_band/1/0.4111378623/0.0000000000/0.0
13/min/i_band/1/0.4111378623/0.0000000000/25.0
13/min/i_band/1/0.4111378623/0.3926990817
13/min/i_band/1/0.4111378623/0.3926990817/0.0
13/min/i_band/1/0.4111378623/0.3926990817/25.0
13/min/i_band/1/0.4111378623/0.7853981634
13/min/i_band/1/0.4111378623/0.7853981634/0.0
13/min/i_band/1/0.4111378623/0.7853981634/0.20
13/min/i_band/1/0.4111378623/0.7853981634/0.20/0
13/min/i_band/1/0.4111378623/0.7853981634/0.20/0/estimate
13/min/i_band/1/0.4111378623/0.7853981634/0.20/0/eval_points0
13/min/i_band/1/0.4111378623/0.7853981634/0.20/0/eval_points1
13/min/i_band/1/0.4111378623/0.7853981634/0.20/25
13/min/i_band/1/0.4111378623/0.7853981634/0.20/25/estimate
13/min/i_band/1/0.4111378623/0.7853981634/0.20/25/eval_points0
13/min/i_band/1/0.4111378623/0.7853981634/0.20/25/eval_points1
13/min/i_band/1/0.4111378623/0.7853981634/25.0
13/min/i_band

In [17]:
for path in DM_path_list:
    clstNo = path.split('/')[0]
    DM_clst_dict = \
        compDist.combine_DM_df_and_h5_to_dict(
            DM_gpBy, DM_fhat, path)
    star_dict = compDist.get_star_dict_from_run(
        star_df, star_fhat, path, star_gpBy_key_no)
    plotDM.plot_DM_fhat(
        fhat=DM_clst_dict, fhat_stars=star_dict, clstNo=clstNo)
    plt.show()
    plt.close()

ValueError: must supply a a same-length tuple to get_group with multiple grouping keys

> /Users/karenyng/Documents/illustris_analyses/code/IlEnv/lib/python2.7/site-packages/pandas/core/groupby.py(439)_get_index()
    437                     msg = ("must supply a a same-length tuple to get_group"
    438                            " with multiple grouping keys")
--> 439                     raise ValueError(msg)
    440 
    441             name = tuple([ convert(n, k) for n, k in zip(name,sample) ])

ipdb> up
> /Users/karenyng/Documents/illustris_analyses/code/IlEnv/lib/python2.7/site-packages/pandas/core/groupby.py(597)get_group()
    595             obj = self._selected_obj
    596 
--> 597         inds = self._get_index(name)
    598         return obj.take(inds, axis=self.axis, convert=False)
    599 

ipdb> up
> /Users/karenyng/Documents/illustris_analyses/code/compute_distance.py(165)combine_DM_df_and_h5_to_dict()
    163     gpbykeys = DM_h5path_to_groupbykey(h5path)
    164     # May want to convert some of the quantities back to float!?
--> 165     fhat =

In [18]:
DM_gp_keys

('peaks_dens',
 'clstNo',
 'cut',
 'weights',
 'los_axis',
 'xi',
 'phi',
 'sig_fraction',
 'kernel_width')

In [19]:
DM_df.keys()[-9:]

Index([u'peaks_dens', u'clstNo', u'cut', u'weights', u'los_axis', u'xi', u'phi', u'sig_fraction', u'kernel_width'], dtype='object')